# Repository Demo



Here is a flowchart for the actions that the tool takes.

Each of the following blocks are described in more detail through this notebook. Feel free to change parameters and experiment!

<p align="center">
  <img src="figures/drawio_flowchart.png" />
</p>

# 1. Getting Started

## 1.1 Ensure setup is accurate

### 1.1.1 System Requirements for TRELPy
- OS: Ubuntu (Has been tested to work on Ubuntu 20.04)
- Computer with GPU to run inference


### 1.1.2. Uncomment and run the following scripts to install all the pip and apt dependencies
```bash
pip3 install -r pip-requirements.txt &&
sudo apt-get -y update &&
sed 's/#.*//' apt-requirements.txt | xargs sudo apt-get -y install
```


### 1.1.3. Ensure all the following programs are installed.

|Name and link of program|What kind of installation| Versions tested on |
|-|-|-|
| [CUDA](https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html) | Local, Docker | cuda_12.4.r12.4 |
| [STORM](https://www.stormchecker.org/documentation/obtain-storm/build.html) | Local | - |
| [PyTorch](https://pytorch.org/get-started/locally/) | Local | 2.1.0+cu12 |
| [StormPy](https://moves-rwth.github.io/stormpy/installation.html) | Local |  |
| [TuLiP](https://github.com/tulip-control/tulip-control) |  Local |  |
| [MMDetection3D](https://mmdetection3d.readthedocs.io/en/latest/get_started.html) | Local |  |
| [NVIDIA Container Toolkit](https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/latest/install-guide.html) | Local, Docker | 1.14.6 |
| [PRISM (Optional)](https://www.prismmodelchecker.org/manual/InstallingPRISM/Instructions) | Local |  |

**Local** means you are running on your ubuntu installation \
**Docker** means you will be using the provided Dockerfile. *This is currently a work in progress and not completely setup.*

### 1.1.4 Testing installation validity

In [2]:
import os
import sys
import subprocess
import numpy as np
import pickle as pkl
from pathlib import Path
from datetime import datetime
from pyquaternion import Quaternion
from itertools import chain, combinations
from typing import Tuple, Dict, Any, List
from confusion_matrix import ConfusionMatrix
from generate_confusion_matrix import GenerateConfusionMatrix

import torch

from nuscenes import NuScenes
from nuscenes.eval.common.config import config_factory
from nuscenes.eval.common.data_classes import EvalBoxes

ImportError: cannot import name 'dataset_version' from partially initialized module 'config' (most likely due to a circular import) (/home/ranai/nuscenes_dataset/3D_Detection/config.py)

## 1.2. Setting up datasets

The NuScenes dataset can be downloaded from [this link after logging in](https://www.nuscenes.org/nuscenes#download:~:text=Show%20more%20%E2%86%93-,Downloads,-Here%20we%20list). \
Instructions for setting up nuscenes for working with MMDetection3D can be found at [MMDetection3D Dataset Preperation](https://mmdetection3d.readthedocs.io/en/latest/user_guides/dataset_prepare.html)

## 1.3. Getting setup for running inference

In this step, you will download and install a model so that you can begin running inference on the dataset you downloaded. \
\
**Config File** is a python file that contains parameters such as batch size, list of classes, indices, input size, etc.   
**Checkpoint File** is a `.pth` file which contains a the exact values of all parameters (weights, current learning rate, etc.) and stores all of this in non-volatile memory.

| Model Name | Modality |Link to Checkpoint file | Link to Config file | mAP (%) | Accuracy (%) | Link to paper |
|-|-|-|-|-|-|-|
|NuScenes SECFPN|Lidar|[Backbone file](https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d/hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d_20210826_225857-f19d00a3.pth)|[Config File](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_secfpn_sbn-all_8xb4-2x_nus-3d.py)|34.33|49.1|[PointPillars](https://arxiv.org/abs/1812.05784)|
|NuScenes SECFPN(FP16)|Lidar|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_secfpn_sbn-all_8xb2-amp-2x_nus-3d.py)|[Config file](https://download.openmmlab.com/mmdetection3d/v0.1.0_models/fp16/hv_pointpillars_secfpn_sbn-all_fp16_2x8_2x_nus-3d/hv_pointpillars_secfpn_sbn-all_fp16_2x8_2x_nus-3d_20201020_222626-c3f0483e.pth)|35.19|50.27|[PointPillars](https://arxiv.org/abs/1812.05784)|
|NuScenes FPN|Lidar|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py)|[Config File](https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth)|39.7|53.2|[PointPillars](https://arxiv.org/abs/1812.05784)|
|NuScenes FPN (FP16)|Lidar|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb2-amp-2x_nus-3d.py)|[Config file](https://download.openmmlab.com/mmdetection3d/v0.1.0_models/fp16/hv_pointpillars_fpn_sbn-all_fp16_2x8_2x_nus-3d/hv_pointpillars_fpn_sbn-all_fp16_2x8_2x_nus-3d_20201021_120719-269f9dd6.pth)|39.2|53.2|[PointPillars](https://arxiv.org/abs/1812.05784)|
|-|-|-|-|-|-|-|
|BEVFusion|Lidar + Camera|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/fe25f7a51d36e3702f961e198894580d83c4387b/projects/BEVFusion/configs/bevfusion_lidar_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d.py)|[Config file](https://download.openmmlab.com/mmdetection3d/v1.1.0_models/bevfusion/bevfusion_lidar_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d-2628f933.pth)|69.6|64.9|[BEVFusion](https://arxiv.org/abs/2205.13542)|

## 1.4 Setup your custom environment

The following cell contains content of the file `custom_env.py`. This is a configuration file that stores path variables, code parameters, etc. Once you fill out the following cell and run this notebook to ensure accuracy of this file, move the contents of this file to `custom_env.py`.

In [ ]:
######## PARMS #########
## Inference model params ##
model_name = "model2_good"  # The name of the directory where the ML model for inference is stored
modality = "lidar"          # The modality of the data
is_mini = True              # Are you using this on NuScenes Mini?

## Confusion Matrix Generation Params ##
verbose = True
###### PARAMS END ######


####### Configuring the right dataset ########
# The code looks in mmdetection3d/data/ for a dataset folder or symlink called `dataset` to find a dataset with size `size`.
# The results will be stored in inside a folder titled `inference_results_path`
if is_mini:
    dataset = "nuscenes-mini"   
    size = "mini"
else:
    dataset = "nuscenes-full"
    size= "full"
    
########### METHODS #############
def getGitRoot():
    """Gets the root directory of the git repository

    Returns:
        str: path the denotes the root directory of the git repository
    """
    return subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

def create_dir_if_not_exist(dir_path):
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} not found. Creating...")
        os.makedirs(dir_path)
    else:
        print(f"Not creating {dir_path} because it already exists")

def is_set_to_mini():
    return is_mini
###### METHODS END #########


home_dir = str(Path.home())
repo_dir = f"{home_dir}/nuscenes_dataset/3D_Detection"  #................# The directory where the repo is stored
dataset_root = f"{home_dir}/software/mmdetection3d/data/{dataset}/"  #...# The directory where the dataset is stored
output_dir = f"{home_dir}/inference_results/{dataset}/{model_name}" #..............# The directory where the output of inference will be stored
model_dir  = f"{output_dir}/{model_name}" #..............................# The directory where the inference model is stored
preds_dir  = f"{model_dir}/preds" #......................................# The directory where inference predictions are stored
cm_dir = f"{repo_dir}/saved_cms/{modality}/{size}/{model_name}" #........# The directory where the confusion matrices generated by the tool will be stored 
create_dir_if_not_exist(cm_dir)

###########################
### Standard Parameters ###
eval_set_map = {
        'v1.0-mini': 'mini_val',
        'v1.0-trainval': 'val',
        'v1.0-test': 'test'
    }

dataset_version = 'v1.0-mini' if is_set_to_mini() else 'v1.0-trainval'

try:
    eval_version = 'detection_cvpr_2019'
    eval_config = config_factory(eval_version)
except:
    eval_version = 'cvpr_2019'
    eval_config = config_factory(eval_version)

## 1.4. Ensure NuScenes is setup correctly

In [ ]:

############# IMPORTS #############
## Usually you would use the following imports to get all necessary paths from the custom_env.py file

# from custom_env import dataset_root as dataroot
# from custom_env import cm_dir, model_dir, eval_version, eval_config
# from custom_env import is_set_to_mini, eval_set_map, dataset_version, eval_version 
##################################

# parameters to setup nuScenes

nusc = NuScenes(version=dataset_version, dataroot = dataset_root)

# 2. Run Inference

In [ ]:
## To run inference, change line 3 to `if True:`

if False:
    now = datetime.now()
    configs_path = "configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py"
    checkpoint_path = "checkpoints/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth"

    folder_name = "model_"+now.strftime("%m-%d-%Y_%H_%M")
    out_dir = f"{output_dir}/" + folder_name

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    info_file = os.path.join(out_dir, "model_info.txt")
    with open(info_file, 'w') as f:
        f.write(f"configs_path = {configs_path} \n checkpoint_path = {checkpoint_path} \n")
    f.close()
        
    pcd_path = f"{dataset_root}/samples/LIDAR_TOP/"

    pcd_list = os.listdir(pcd_path)
    print(len(pcd_list))

    for i, pcd in enumerate(pcd_list):
        path = Path(f"{pcd_path}/{pcd}").absolute()
        if path.exists():
            cmd = f'python3 demo/pcd_demo.py {str(path)} {configs_path} {checkpoint_path} --device cuda --out-dir {out_dir}'
        
        ##### Uncomment this to run the inference ######    
        subprocess.run(cmd, cwd=f"{home_dir}/software/mmdetection3d/", shell=True)
        
        if i%100 == 0:
            print(f"---- ---- !-!-!-!- run_inference.py: Done with {i} files")

    with open(info_file, 'a') as f:
        f.write(f"Inferences complete.")
    f.close()

    print(f"Inference complete. Output written to {out_dir}")

# 3. Confusion Matrix Generation

## 3.1 Setup 

In the following block, there are various variables that you can change to change the behavior of the Confusion Matrix Generation

| Variable name | Type | Description |
|--|--|--|
| `list of classes` | `list` | The class labels for the confsion matrix |
|`conf_mat_mapping`|`dict`| Dict ***keys*** represent output classes for inference |
|`conf_mat_mapping`|`dict`| Dict ***values*** represent the class lable to match it with|
| `labels` | `dict` | Dict ***keys*** represent place in the confusion matrix |
| `labels` | `dict` | Dict ***values*** represent place in the confusion matrix   |

<p align="center">
    <img src="figures/Distance_param.jpg" width=550px height=600px>
</p>

In [ ]:
list_of_classes = ["ped", "obs"]        # The classes that are to be considered for the confusion matrix

PED = 0
OBS = 1
EMPTY = 2
                                        # TODO
labels = {0: "ped", 1: "obs", 2:"empty"}

conf_mat_mapping = {                    # The mapping from the output of the model to the classes in the confusion matrix
    "pedestrian": PED,
    "bus": OBS,
    "car" : OBS,
    "truck": OBS,
    "bicycle": OBS,
    "motorcycle": OBS,
    "traffic_cone": OBS
}

In [ ]:
generator = GenerateConfusionMatrix(nusc=nusc,      
    config=eval_config,
    result_path=f'{model_dir}/results_nusc.json',   ## PARAM Where are the results are stored
    eval_set=eval_set_map[dataset_version],
    output_dir=os.getcwd(), #.......................## PARAM Where to store the output
    verbose=verbose,  #.............................## PARAM Verbose
    conf_mat_mapping=conf_mat_mapping,
    list_of_classes=list_of_classes,
    distance_parametrized=True,
    max_dist=100, #................................## PARAM The maximum distance the model considers
    distance_bin=10 #..............................## PARAM For distance parametrized confusion matrices, the distance between radius bands
)

## 3.2 Visualizing the scene

In [ ]:
####### RENDERING LIBRARIES ######

from nuscenes_render import render_sample_data_with_predictions

In [ ]:
import random

random.seed = 42

sample_tokens = generator.gt_boxes.sample_tokens # All the sample tokens in the dataset

tok = random.choice(sample_tokens)
sample_data_token = nusc.get('sample', tok)['data']['LIDAR_TOP']

print(f"--------- Details for sample {tok} ------------")
print(f"Number of ground truth objects {len(generator.gt_boxes[tok])}")
print(f"Number of prediction objects {len(generator.pred_boxes[tok])}")

render_sample_data_with_predictions(nusc=nusc, 
                                    sample_token=sample_data_token, 
                                    gt_boxes=generator.gt_boxes[tok], 
                                    pred_boxes=generator.pred_boxes[tok], 
                                    verbose=True)



## Setup for probability plot generation



In [ ]:
#TODO Does the env have a ped? empty? AND the user can set VMAX, Ncar 
# We generate probabilitie based on that
# cb_cm_ped

# Bibliography

```latex
@inproceedings{PointPillars,
  title={Pointpillars: Fast encoders for object detection from point clouds},
  author={Lang, Alex H and Vora, Sourabh and Caesar, Holger and Zhou, Lubing and Yang, Jiong and Beijbom, Oscar},
  booktitle={Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition},
  pages={12697--12705},
  year={2019}
}
```

```latex
@inproceedings{BevFusion,
  title={BEVFusion: Multi-Task Multi-Sensor Fusion with Unified Bird's-Eye View Representation},
  author={Liu, Zhijian and Tang, Haotian and Amini, Alexander and Yang, Xingyu and Mao, Huizi and Rus, Daniela and Han, Song},
  booktitle={IEEE International Conference on Robotics and Automation (ICRA)},
  year={2023}
}
```